# 📦 Data Notebook — Newegg Price Tracker (Catégorie entière)

Ce notebook regroupe **tout ce qui concerne la partie data** du projet :
- description du dataset (`prices_history.csv`)
- schéma des colonnes (data dictionary)
- gestion des valeurs manquantes
- nettoyage et préparation
- analyses simples (EDA)
- indicateurs journaliers + moyenne mobile
- exports recommandés

> **Fichier attendu** : `prices_history.csv` dans le même dossier que ce notebook.


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

CSV_PATH = "prices_history.csv"

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"⚠️ Fichier introuvable: {CSV_PATH}. Place-le à côté du notebook.")

df_raw = pd.read_csv(CSV_PATH, encoding="utf-8-sig")
df_raw.head(5)


FileNotFoundError: ⚠️ Fichier introuvable: prices_history.csv. Place-le à côté du notebook.

## 1) Data dictionary (schéma des colonnes)

Le dataset provient du scraping Newegg. Chaque ligne représente **un produit observé à une date/heure de collecte**.

| Colonne | Type attendu | Description |
|---|---|---|
| `category` | texte | Catégorie suivie (ex: GPU, SSD…) |
| `scrape_datetime_utc` | datetime ISO | Date/heure de collecte en UTC |
| `page` | int | Numéro de page (pagination) |
| `rank` | int | Rang du produit dans la liste (utile pour analyser le “Top”) |
| `product_name` | texte | Nom du produit |
| `product_url` | texte (URL) | Identifiant stable du produit (clé principale) |
| `brand` | texte | Marque (si détectée) |
| `price_value` | float | Prix numérique (utile pour analyses) |
| `rating_avg` | float | Note moyenne (si disponible) |
| `rating_count` | int | Nombre d’avis (si disponible) |
| `availability` | texte | Stock (In stock / Out of stock / Unknown) |


In [ ]:
# 2) Aperçu global : dimensions, types, exemples
print("Shape:", df_raw.shape)
display(df_raw.sample(min(5, len(df_raw)), random_state=42))

print("\nTypes:")
display(df_raw.dtypes)

print("\nInfo:")
df_raw.info()


In [ ]:
# 3) Valeurs manquantes
missing = df_raw.isna().sum().sort_values(ascending=False)
missing_pct = (missing / len(df_raw) * 100).round(2)

miss_table = pd.DataFrame({"missing_count": missing, "missing_pct": missing_pct})
display(miss_table)


## 4) Nettoyage & préparation

Objectif : rendre les données **analysables**.

Étapes :
1. Convertir `scrape_datetime_utc` → datetime UTC  
2. Extraire `scrape_date` (jour) pour analyses journalières  
3. Convertir `price_value` → numérique  
4. Nettoyer les champs texte (`strip`)  
5. Filtrer les lignes inutilisables (pas d’URL, pas de date, pas de prix valide)  
6. Dédoublonner strictement sur (`product_url`, `scrape_datetime_utc`)  


In [ ]:
df = df_raw.copy()

# Champs texte
for col in ["category", "product_name", "product_url", "brand", "availability"]:
    if col in df.columns:
        df[col] = df[col].astype("string").fillna("").str.strip()

# Date/heure
df["scrape_datetime_utc"] = pd.to_datetime(df["scrape_datetime_utc"], errors="coerce", utc=True)
df["scrape_date"] = df["scrape_datetime_utc"].dt.date

# Numériques
df["price_value"] = pd.to_numeric(df["price_value"], errors="coerce")
if "rating_avg" in df.columns:
    df["rating_avg"] = pd.to_numeric(df["rating_avg"], errors="coerce")
if "rating_count" in df.columns:
    df["rating_count"] = pd.to_numeric(df["rating_count"], errors="coerce")

# Filtres qualité
df = df.dropna(subset=["scrape_datetime_utc", "scrape_date"])
df = df[df["product_url"].str.len() > 0]
df = df[df["price_value"].notna() & (df["price_value"] > 0)]

# Dédoublonnage exact (tracker)
df = df.sort_values(["product_url", "scrape_datetime_utc"])
df = df.drop_duplicates(subset=["product_url", "scrape_datetime_utc"], keep="last")

print("✅ Après nettoyage:", df.shape)
df.head(5)


In [ ]:
# 5) Catégories disponibles + période couverte
print("Catégories:", df["category"].unique().tolist())

date_min = df["scrape_datetime_utc"].min()
date_max = df["scrape_datetime_utc"].max()
print("Période:", date_min, "->", date_max)

# Nombre de produits distincts par catégorie
display(df.groupby("category")["product_url"].nunique().sort_values(ascending=False))


## 6) EDA (analyses simples)

Analyses proposées :
- distribution des prix
- top 10 produits les moins chers / les plus chers
- prix moyen par marque
- disponibilité (stock)
- détection simple d’outliers (IQR)


In [ ]:
CATEGORY = df["category"].iloc[0] if len(df) else None  # par défaut, 1ère catégorie trouvée
print("Catégorie par défaut:", CATEGORY)

d = df[df["category"] == CATEGORY].copy()
print("Lignes catégorie:", len(d))

# Résumé prix
summary = d["price_value"].describe(percentiles=[.25, .5, .75]).to_frame("price_value")
display(summary)

# Top 10 moins chers / plus chers
display(d.sort_values("price_value").head(10)[["product_name","brand","price_value","availability"]])
display(d.sort_values("price_value", ascending=False).head(10)[["product_name","brand","price_value","availability"]])


In [ ]:
# Prix moyen par marque (top 15 marques avec le plus de produits)
brand_counts = d["brand"].replace("", "Unknown").value_counts()
top_brands = brand_counts.head(15).index.tolist()

brand_avg = (
    d.assign(brand=d["brand"].replace("", "Unknown"))
     .query("brand in @top_brands")
     .groupby("brand")
     .agg(avg_price=("price_value","mean"), n_products=("product_url","nunique"))
     .sort_values("avg_price")
)

display(brand_avg)

# Disponibilité
avail = d["availability"].replace("", "Unknown").value_counts()
display(avail)


In [ ]:
# Graphiques simples (sans seaborn)

# 1) Histogramme des prix (distribution)
plt.figure()
d["price_value"].plot(kind="hist", bins=30)
plt.title(f"Distribution des prix — {CATEGORY}")
plt.xlabel("Prix")
plt.ylabel("Nombre de produits")
plt.tight_layout()
plt.show()

# 2) Bar chart prix moyen par marque (top_brands)
plt.figure()
brand_avg["avg_price"].plot(kind="bar")
plt.title(f"Prix moyen par marque — {CATEGORY} (Top 15 marques par volume)")
plt.xlabel("Marque")
plt.ylabel("Prix moyen")
plt.tight_layout()
plt.show()


In [ ]:
# Détection simple des outliers (méthode IQR)
q1 = d["price_value"].quantile(0.25)
q3 = d["price_value"].quantile(0.75)
iqr = q3 - q1

lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

outliers = d[(d["price_value"] < lower) | (d["price_value"] > upper)]
print("Outliers:", len(outliers), "sur", len(d), f"({len(outliers)/len(d)*100:.2f}%)")

display(outliers.sort_values("price_value", ascending=False).head(10)[["product_name","brand","price_value","product_url"]])


## 7) Statistiques journalières + moyenne mobile 7 jours

Quand tu auras plusieurs collectes (plusieurs jours), cette partie donnera :
- prix moyen/jour
- médiane/jour
- nombre de produits distincts/jour
- moyenne mobile 7 jours (tendance)


In [ ]:
daily = (
    d.groupby("scrape_date")
     .agg(
        avg_price=("price_value","mean"),
        median_price=("price_value","median"),
        min_price=("price_value","min"),
        max_price=("price_value","max"),
        products_count=("product_url","nunique"),
        observations=("price_value","count"),
     )
     .reset_index()
     .sort_values("scrape_date")
)

daily["avg_price_ma7"] = daily["avg_price"].rolling(window=7, min_periods=1).mean()

display(daily)

# Plot
plt.figure()
x = pd.to_datetime(daily["scrape_date"])
plt.plot(x, daily["avg_price"], label="Prix moyen (jour)")
plt.plot(x, daily["avg_price_ma7"], label="Moyenne mobile 7 jours")
plt.title(f"Prix moyen — {CATEGORY}")
plt.xlabel("Date")
plt.ylabel("Prix")
plt.legend()
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()


## 8) Top baisses (produits) entre première et dernière observation

⚠️ Si tu n’as qu’une seule date de collecte, la table sera vide (normal).  
Quand tu auras plusieurs collectes, cette table permettra d’identifier les produits ayant le plus baissé.


In [ ]:
if d["scrape_date"].nunique() < 2:
    print("ℹ️ Une seule date de collecte => pas de comparaison possible pour les baisses.")
else:
    tmp = d.sort_values(["product_url","scrape_datetime_utc"])

    first = (
        tmp.groupby("product_url", as_index=False)
           .first()[["product_url","product_name","scrape_date","price_value"]]
           .rename(columns={"scrape_date":"date_first","price_value":"price_first"})
    )
    last = (
        tmp.groupby("product_url", as_index=False)
           .last()[["product_url","scrape_date","price_value"]]
           .rename(columns={"scrape_date":"date_last","price_value":"price_last"})
    )

    drops = first.merge(last, on="product_url", how="inner")
    drops["drop_abs"] = drops["price_first"] - drops["price_last"]
    drops["drop_pct"] = (drops["drop_abs"] / drops["price_first"]) * 100
    drops = drops[drops["drop_abs"] > 0].sort_values("drop_pct", ascending=False)

    display(drops.head(15))


## 9) Exports (optionnel)

Tu peux exporter :
- `clean_prices_history.csv` : données nettoyées
- `daily_stats.csv` : stats journalières de la catégorie


In [ ]:
# Export optionnel
df.to_csv("clean_prices_history.csv", index=False, encoding="utf-8-sig")
daily.to_csv(f"daily_stats_{CATEGORY.lower()}.csv", index=False, encoding="utf-8-sig")

print("✅ Exports créés: clean_prices_history.csv, daily_stats_<category>.csv")
